# Set up

### Import Python packages to use in the script

In [1]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
import skan

### Define the functions that will run different steps/tasks on the data

Need to select which GPU device to send data to

In [2]:
cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              573.24
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

In [18]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [38]:
def get_measurements(mask,img,props):
    filtered_masks = sk.segmentation.clear_border(mask)
    df = sk.measure.regionprops_table(filtered_masks,img,properties=props)
    df = pd.DataFrame.from_dict(df)
    return filtered_masks, df

In [20]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [30]:
all_img[0].shape

(11, 2048, 2048)

In [48]:
def get_3D_surfacearea_and_sphericity(mask_img,scaled_df):
    surf_area_list = []
    sphiricity_list = []
    vol = np.asarray(scaled_df['Volume (um^3)']).astype(np.float64)
    c = 0
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=(voxel_size_z,voxel_size_x,voxel_size_y))
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphiricity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphiricity_list.append(sphiricity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphiricities = pd.Series(sphiricity_list,name='Sphericity')
    merged_df = pd.concat([scaled_df,surface_areas,sphiricities], axis=1)
    return merged_df

In [ ]:
def sphericity(mesh_volume, mesh_surface_area):
    
    '''This definition of sphericity assumes that you are working in continuous space.
    Parameters:
    -----------
    mesh_volume: integer or float value
    mesh_surface_area: integer or float value
    Returns:
    --------
    psi: a float value with range(0,1) reflecting the compactness of an object
    Got this script from: https://github.com/BiAPoL/Bio-image_Analysis_with_Python/blob/main/05_feature_extraction/02_sphericity_and_solidity.ipynb
    '''
    numerator = (np.pi ** (1/3)) * ((6 * mesh_volume) ** (2/3))
    denominator = mesh_surface_area
    
    psi = numerator / denominator
    
    return psi

In [22]:
def save(save_path, img_name, norm_img, filtered_masks, merged_df):
    # make directories if they do not exist
    try:
        norm_path = os.mkdir(os.path.join(save_path,'norm_imgs'))
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        norm_path = os.path.join(save_path,'norm_imgs')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    imsave(os.path.join(norm_path,'normalized_'+img_name[:-4]+'.tif'),norm_img)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Load in Data

## Two options for loading in data:

### Option 1: Specify the locations of all image containing folders:
Then merge into a long list of images and file names

In [ ]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [ ]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [ ]:
all_files = CMO_files+GMO_1ng_files+GMO_2ng_files # need this for getting image names later
all_img = CMO_imgs+GMO_1ng_imgs+GMO_2ng_imgs

### Option 2: Use a recursive search option to find all images in the parent directory:

In [23]:
all_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/**/*.tif',recursive=True)) #finds all tiff images in the subdirectories of TIF_Files
all_img = list(map(sk.io.imread,all_files))

# Run Pipeline

### Optional use of random integer generator if you want to test on single random images first

In [5]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))

In [8]:
tophat_radius = 10
img = normalize_images(all_img[i],tophat_radius) #change the tophat_radius to increase or decrease the background as needed

In [9]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(img, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

creating new log file
2025-05-23 15:42:32,233 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-05-23 15:42:32,234 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-05-23 15:42:32,236 [INFO] ** TORCH CUDA version installed and working. **
2025-05-23 15:42:32,237 [INFO] >>>> using GPU (CUDA)
2025-05-23 15:42:32,238 [INFO] >> cyto3 << model set to be used
2025-05-23 15:42:32,344 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-05-23 15:42:32,607 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-05-23 15:42:32,609 [INFO] channels set to [[0, 0]]
2025-05-23 15:42:32,610 [INFO] ~~~ FINDING MASKS ~~~
2025-05-23 15:42:32,612 [INFO] multi-stack tiff read in as having 11 planes 1 channels
2025-05-23 15:42:36,374 [INFO] 100%|##########| 11/11 [00:03<00:00,  3.49it/s]
2025-05-23 15:42:36,780 [INFO] network run in 3.56s
2025-05-23 15:42:

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.78it/s]


2025-05-23 15:42:45,609 [INFO] masks created in 8.83s
2025-05-23 15:42:50,038 [INFO] >>>> TOTAL TIME 17.43 sec


In [10]:
#View results of segmentation in Napari
viewer = napari.view_image(all_img[i], name='original img', scale = (4.55,0.3,0.3))
viewer.add_image(img,name='nomarlized img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

<Image layer 'masks' at 0x1c80d5e5d50>

### Run pipeline on all images

Set up a few parameters before running

Properties that can be measured with scikit-image can be found here: [Scikit-image Region Properties](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops)

In [33]:
tophat_radius = 10
voxel_size_x = 0.301
voxel_size_y = 0.301
voxel_size_z = 4.55
voxel = 0.301*0.301*4.55
save_path = 'E:/ParadaKusz_Lab/Test_Outputs/'
props = ('label','area','intensity_mean','intensity_min','intensity_max')

In [25]:
norm_imgs = [normalize_images(img,tophat_radius) for img in all_img]

In [27]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(norm_imgs, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

creating new log file
2025-05-23 16:22:27,176 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-05-23 16:22:27,177 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-05-23 16:22:27,180 [INFO] ** TORCH CUDA version installed and working. **
2025-05-23 16:22:27,181 [INFO] >>>> using GPU (CUDA)
2025-05-23 16:22:27,182 [INFO] >> cyto3 << model set to be used
2025-05-23 16:22:27,301 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-05-23 16:22:27,596 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-05-23 16:22:27,596 [INFO] channels set to [[0, 0]]
2025-05-23 16:22:27,596 [INFO] ~~~ FINDING MASKS ~~~
2025-05-23 16:22:27,596 [INFO] 0%|          | 0/25 [00:00<?, ?it/s]
2025-05-23 16:22:27,596 [INFO] multi-stack tiff read in as having 11 planes 1 channels
2025-05-23 16:22:28,189 [INFO] 
2025-05-23 16:22:28,189 [INFO] 0%|          | 0/


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 63.56it/s]


2025-05-23 16:22:36,909 [INFO] masks created in 5.58s
2025-05-23 16:22:41,363 [INFO] multi-stack tiff read in as having 32 planes 1 channels
2025-05-23 16:22:43,218 [INFO] 
2025-05-23 16:22:43,220 [INFO] 0%|          | 0/32 [00:00<?, ?it/s]
2025-05-23 16:22:43,220 [INFO] 
2025-05-23 16:22:50,344 [INFO] 100%|##########| 32/32 [00:07<00:00,  4.49it/s]
2025-05-23 16:22:51,379 [INFO] network run in 8.16s
2025-05-23 16:22:51,381 [INFO] 
2025-05-23 16:22:51,384 [INFO] 0%|          | 0/32 [00:00<?, ?it/s]
2025-05-23 16:22:51,387 [INFO] 
2025-05-23 16:23:17,312 [INFO] 100%|##########| 32/32 [00:25<00:00,  1.23it/s]
2025-05-23 16:23:17,312 [INFO] stitching 32 planes using stitch_threshold=0.200 to make 3D masks



100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:01<00:00, 28.05it/s]


2025-05-23 16:23:20,364 [INFO] masks created in 28.98s
2025-05-23 16:23:33,219 [INFO] 8%|8         | 2/25 [01:05<12:34, 32.81s/it]
2025-05-23 16:23:33,219 [INFO] multi-stack tiff read in as having 35 planes 1 channels
2025-05-23 16:23:35,161 [INFO] 
2025-05-23 16:23:35,161 [INFO] 0%|          | 0/35 [00:00<?, ?it/s]
2025-05-23 16:23:35,161 [INFO] 
2025-05-23 16:23:42,933 [INFO] 100%|##########| 35/35 [00:07<00:00,  4.50it/s]
2025-05-23 16:23:44,156 [INFO] network run in 8.99s
2025-05-23 16:23:44,156 [INFO] 
2025-05-23 16:23:44,156 [INFO] 0%|          | 0/35 [00:00<?, ?it/s]
2025-05-23 16:23:44,156 [INFO] 
2025-05-23 16:24:14,240 [INFO] 
2025-05-23 16:24:14,243 [INFO] 97%|#########7| 34/35 [00:30<00:00,  1.13it/s]
2025-05-23 16:24:14,245 [INFO] 
2025-05-23 16:24:14,953 [INFO] 100%|##########| 35/35 [00:30<00:00,  1.14it/s]
2025-05-23 16:24:14,953 [INFO] stitching 35 planes using stitch_threshold=0.200 to make 3D masks


...

2025-05-23 16:46:34,965 [INFO] masks created in 40.89s
2025-05-23 16:46:51,520 [INFO] 92%|#########2| 23/25 [24:23<02:19, 69.81s/it]
2025-05-23 16:46:51,522 [INFO] multi-stack tiff read in as having 66 planes 1 channels
2025-05-23 16:46:54,961 [INFO] 
2025-05-23 16:46:54,963 [INFO] 0%|          | 0/66 [00:00<?, ?it/s]
2025-05-23 16:46:54,965 [INFO] 
2025-05-23 16:47:09,092 [INFO] 100%|##########| 66/66 [00:14<00:00,  4.67it/s]
2025-05-23 16:47:11,161 [INFO] network run in 16.20s
2025-05-23 16:47:11,163 [INFO] 
2025-05-23 16:47:11,164 [INFO] 0%|          | 0/66 [00:00<?, ?it/s]
2025-05-23 16:47:11,165 [INFO] 
2025-05-23 16:47:41,313 [INFO] 
2025-05-23 16:47:41,315 [INFO] 58%|#####7    | 38/66 [00:30<00:22,  1.26it/s]
2025-05-23 16:47:41,317 [INFO] 
2025-05-23 16:47:55,404 [INFO] 
2025-05-23 16:47:55,405 [INFO] 58%|#####7    | 38/66 [00:44<00:22,  1.26it/s]
2025-05-23 16:47:55,406 [INFO] 
2025-05-23 16:48:01,868 [INFO] 100%|##########| 66/66 [00:50<00:00,  1.30it/s]
2025-05-23 16:48:01,8


100%|█████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:02<00:00, 25.93it/s]


2025-05-23 16:48:08,134 [INFO] masks created in 56.97s
2025-05-23 16:48:33,006 [INFO] 96%|#########6| 24/25 [26:05<01:19, 79.31s/it]
2025-05-23 16:48:33,008 [INFO] multi-stack tiff read in as having 52 planes 1 channels
2025-05-23 16:48:35,930 [INFO] 
2025-05-23 16:48:35,932 [INFO] 0%|          | 0/52 [00:00<?, ?it/s]
2025-05-23 16:48:35,934 [INFO] 
2025-05-23 16:48:46,930 [INFO] 100%|##########| 52/52 [00:10<00:00,  4.73it/s]
2025-05-23 16:48:48,645 [INFO] network run in 12.72s
2025-05-23 16:48:48,647 [INFO] 
2025-05-23 16:48:48,648 [INFO] 0%|          | 0/52 [00:00<?, ?it/s]
2025-05-23 16:48:48,650 [INFO] 
2025-05-23 16:49:18,694 [INFO] 
2025-05-23 16:49:18,696 [INFO] 63%|######3   | 33/52 [00:30<00:17,  1.10it/s]
2025-05-23 16:49:18,699 [INFO] 
2025-05-23 16:49:32,199 [INFO] No cell pixels found.
2025-05-23 16:49:32,265 [INFO] 100%|##########| 52/52 [00:43<00:00,  1.19it/s]
2025-05-23 16:49:32,267 [INFO] stitching 52 planes using stitch_threshold=0.200 to make 3D masks



100%|█████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:01<00:00, 28.18it/s]


2025-05-23 16:49:37,587 [INFO] masks created in 48.94s
2025-05-23 16:49:56,814 [INFO] 100%|##########| 25/25 [27:29<00:00, 65.97s/it]
2025-05-23 16:49:56,815 [INFO] >>>> TOTAL TIME 1649.22 sec


In [56]:
for i in range(len(norm_imgs)):
    img_name = os.path.basename(all_files[i])
    print(f'processing {img_name}. {i} of {len(norm_imgs)} complete.')
    mask_img, df = get_measurements(masks[i],all_img[i],props)
    scaled_df = scaled_vol(df,voxel)
    merged_df = get_3D_surfacearea_and_sphericity(mask_img,scaled_df)
    save(save_path, img_name, norm_imgs[i], mask_img, merged_df)

processing Control_MO_00.tif. 0 of 25 complete.
processing Control_MO_01.tif. 1 of 25 complete.
processing Control_MO_02.tif. 2 of 25 complete.
processing Control_MO_03.tif. 3 of 25 complete.
processing Control_MO_04.tif. 4 of 25 complete.
processing Control_MO_05.tif. 5 of 25 complete.
processing Control_MO_06.tif. 6 of 25 complete.
processing Control_MO_07.tif. 7 of 25 complete.
processing gnas_MO_1ng_00.tif. 8 of 25 complete.
processing gnas_MO_1ng_01.tif. 9 of 25 complete.
processing gnas_MO_1ng_02.tif. 10 of 25 complete.
processing gnas_MO_1ng_03.tif. 11 of 25 complete.
processing gnas_MO_1ng_04.tif. 12 of 25 complete.
processing gnas_MO_1ng_05.tif. 13 of 25 complete.
processing gnas_MO_1ng_06.tif. 14 of 25 complete.
processing gnas_MO_1ng_07.tif. 15 of 25 complete.
processing gnas_MO_2ng_00.tif. 16 of 25 complete.
processing gnas_MO_2ng_01.tif. 17 of 25 complete.
processing gnas_MO_2ng_02.tif. 18 of 25 complete.
processing gnas_MO_2ng_03.tif. 19 of 25 complete.
processing gnas_MO